<a href="https://colab.research.google.com/github/samer-glitch/Federated-Governance-and-Provenance-Scoring-for-Trustworthy-AI-A-Metadata-Ledger-Approach/blob/main/Plan_B_5_clients_10_rounds%2C_2_epochs%2C_128_neurons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pandas numpy scikit-learn matplotlib tensorflow "flwr[simulation]==1.18.0" seaborn


In [ ]:
# ──────────────────────────────────────────────────────────────────────────────
# Cell 0: IMPORTS & GLOBAL CONFIGURATION
# ──────────────────────────────────────────────────────────────────────────────
#!pip install -q pandas numpy scikit-learn matplotlib tensorflow "flwr[simulation]==1.18.0" seaborn

import os, io, glob, time, random, uuid, shutil
from datetime import datetime
import matplotlib.dates as mdates

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

import tensorflow as tf
import flwr as fl

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix as sk_confusion_matrix
)

# Global Config
NUM_CLIENTS            = 5      # total distinct data-holding parties that will participate
RANDOM_STATE           = 42     # seed that keeps every shuffle/split reproducible
NUM_ROUNDS_FL          = 10     # how many server-client aggregation cycles the FL run performs
DEFAULT_LOCAL_EPOCHS   = 2      # how many full passes over each client’s local data *per round*
DEFAULT_CLIENT_FRACTION = 1.0   # 1.0 → every round the server selects **all** clients (no sampling)



# Carbon intensity (kg CO2 per kWh) – example global average
CARBON_INTENSITY = 0.475

# Client IDs & version counters
import string, itertools
single = list(string.ascii_uppercase)
double = [''.join(p) for p in itertools.product(string.ascii_uppercase, repeat=2)]
CLIENT_IDS = (single + double)[:NUM_CLIENTS]
version_counters = {}

np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)
tf.keras.utils.set_random_seed(RANDOM_STATE)
tf.get_logger().setLevel("ERROR")
print(f"TensorFlow: {tf.__version__}, Flower: {fl.__version__}")
# ──────────────────────────────────────────────────────────────────────────────
# Cell 1: ONE-TIME LEDGER INITIALIZATION
# ──────────────────────────────────────────────────────────────────────────────
# Wipe old ledgers & create folders
if os.path.exists("./ledgers"):
    shutil.rmtree("./ledgers")
os.makedirs("./ledgers/local", exist_ok=True)
os.makedirs("./ledgers/central", exist_ok=True)
print("⚠️ LEDGERS RESET: All previous ledger files will be deleted now. ⚠️")


# Empty central‐ledger with headers
pd.DataFrame(columns=[
    "tx_id","timestamp","client","version","record_count",
    "dim1_sub","dim2_sub","dim3_sub","dim4_sub","dim5_sub","dim6_sub",
    "pscore","action"
]).to_csv("./ledgers/central/central_ledger.csv", index=False)
# ──────────────────────────────────────────────────────────────────────────────
# Cell 2: DATA LOADING & INITIAL CLEAN-UP
# ──────────────────────────────────────────────────────────────────────────────
from google.colab import files
print("⬆️ Upload diabetes.csv")
uploaded = files.upload()
fname = next(iter(uploaded))
raw_df = pd.read_csv(io.BytesIO(uploaded[fname]))

def preprocess_diabetes(df):
    df = df.copy()
    df.columns = df.columns.str.lower().str.replace('[^a-z0-9]+', '_', regex=True)
    df.replace('?', np.nan, inplace=True)
    if 'age' in df and df['age'].dtype == object:
        df['age'] = df['age'].str.extract('(\d+)').astype(float)
    df['readmitted'] = df['readmitted'].map({'NO':0,'<30':1,'>30':2})
    for col in ['diag_1','diag_2','diag_3']:
        if col in df:
            df[col] = df[col].replace({'V[0-9]{2}': np.nan, 'E[0-9]{3}': np.nan}, regex=True)
    meds = ['metformin','repaglinide','glimepiride','glipizide','glyburide','pioglitazone','insulin']
    for col in meds:
        if col in df:
            df[col] = df[col].replace({'No':0,'Steady':1,'Up':1,'Down':1})
    return df

raw_df = preprocess_diabetes(raw_df)

# Shuffle & split among clients
data_main = raw_df.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)
fracs = list(np.random.dirichlet([1.5]*NUM_CLIENTS))
client_data_splits = {}
start = 0
for cid in CLIENT_IDS:
    if not fracs: break
    frac = fracs.pop()
    end = start + int(frac*len(data_main))
    client_data_splits[cid] = data_main.iloc[start:end].copy()
    start = end
if start < len(data_main):
    client_data_splits[CLIENT_IDS[-1]] = pd.concat([
        client_data_splits[CLIENT_IDS[-1]],
        data_main.iloc[start:]
    ])
print("\nClient Data Distribution:")
for cid, df in client_data_splits.items():
    print(f" • {cid}: {len(df)} samples")
# ──────────────────────────────────────────────────────────────────────────────
# Cell 3: LEAK-PROOF PREPROCESS + CROSS-VALIDATION
# ──────────────────────────────────────────────────────────────────────────────
def preprocess_data(client_data, target_col="readmitted", cv_splits=5):
    # 1) Concatenate and early split
    df_all = pd.concat(client_data.values()).reset_index(drop=True)
    X = df_all.drop(target_col, axis=1)
    y = df_all[target_col]
    X_tr_raw, X_te_raw, y_tr, y_te = train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
    )

    # 2) Impute & Scale / One-Hot
    num_cols = X.select_dtypes(include=np.number).columns.tolist()
    cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
    preprocessor = ColumnTransformer([
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), cat_cols)
    ])
    X_train = preprocessor.fit_transform(X_tr_raw)
    X_test  = preprocessor.transform(X_te_raw)

    # 3) Stratified K-Fold
    cv = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=RANDOM_STATE)

    return X_train, X_test, y_tr, y_te, preprocessor, cv

# Apply to all clients combined
X_all, X_all_test, y_all, y_all_test, preproc_all, cv_all = preprocess_data(client_data_splits)
# ──────────────────────────────────────────────────────────────────────────────
# Cell 4: MODEL ARCHITECTURE, METRICS & 🔍 Neural Network Overview
# ──────────────────────────────────────────────────────────────────────────────

def make_model(input_dim, verbose=True):
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(input_dim,)),
        tf.keras.layers.Dense(
            128,
            activation='relu',
            kernel_initializer=tf.keras.initializers.GlorotUniform(seed=RANDOM_STATE)
        ),
        tf.keras.layers.Dense(
            32,
            activation='relu',
            kernel_initializer=tf.keras.initializers.GlorotUniform(seed=RANDOM_STATE)
        ),
        tf.keras.layers.Dense(
            3,
            activation='softmax',
            kernel_initializer=tf.keras.initializers.GlorotUniform(seed=RANDOM_STATE)
        ),
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(epsilon=1e-7),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    if verbose:
        model.summary()
    return model

def get_detailed_metrics(y_true, y_proba):
    if y_true.size == 0 or y_proba.size == 0:
        return {'accuracy':0,'precision':0,'recall':0,'f1':0,'roc_auc':0}, np.zeros((3,3),int)
    y_pred = np.argmax(y_proba, axis=1)
    cm = sk_confusion_matrix(y_true, y_pred, labels=[0,1,2])
    return {
        'accuracy': accuracy_score(y_true,y_pred),
        'precision': precision_score(y_true,y_pred,average='weighted',zero_division=0),
        'recall': recall_score(y_true,y_pred,average='weighted',zero_division=0),
        'f1': f1_score(y_true,y_pred,average='weighted',zero_division=0),
        'roc_auc': roc_auc_score(y_true, y_proba, multi_class='ovr')
                   if len(np.unique(y_true))>1 else 0
    }, cm

def plot_confusion_matrix(cm, names, title):
    plt.figure(figsize=(7,5))
    sns.heatmap(
        cm, annot=True, fmt='d', cmap='Blues',
        xticklabels=names, yticklabels=names
    )
    plt.title(title)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

def summarize_model(model, input_dim):
    """
    Build the model, then walk through each layer computing its output shape
    via compute_output_shape().
    """
    # Force the model to build so compute_output_shape works
    model.build((None, input_dim))
    rows = []
    current_shape = (None, input_dim)
    for layer in model.layers:
        out_shape = layer.compute_output_shape(current_shape)
        rows.append({
            'Layer': layer.name,
            'Type': layer.__class__.__name__,
            'Output Shape': out_shape,
            'Params': layer.count_params()
        })
        current_shape = out_shape
    return pd.DataFrame(rows)

# Instantiate once on your full feature set
_dummy = make_model(input_dim=X_all.shape[1], verbose=False)

print("\n🔍 Neural Network Overview:")
display(summarize_model(_dummy, X_all.shape[1]))
# ──────────────────────────────────────────────────────────────────────────────
# Cell 5: TADP GOVERNANCE & LEDGER WRITES
# ──────────────────────────────────────────────────────────────────────────────
MEDICAL_CRITICAL_FIELDS = ['age','diag_1','max_glu_serum','a1cresult','readmitted']
WEIGHTS_PSCORE = {
    'dim1':0.25,'dim2':0.15,'dim3':0.10,'dim4':0.10,'dim5':0.30,'dim6':0.10
}

def calculate_medical_quality(df):
    qm={'completeness':0,'errors':0}
    crit=[c for c in MEDICAL_CRITICAL_FIELDS if c in df]
    if crit:
        qm['completeness'] = (1-df[crit].isna().mean().mean())*5
    vc=[]
    if 'diag_1' in df:           vc.append(0.6*df['diag_1'].str.match(r'\d{3}').mean())
    if 'time_in_hospital' in df: vc.append(0.2*(df['time_in_hospital']>=1).mean())
    if 'num_medications' in df:  vc.append(0.2*(df['num_medications']>0).mean())
    if vc: qm['errors']=(1-sum(vc))*5
    return qm

def calculate_usage_constraints(df):
    n=len(df)
    defaults = pd.Series([3]*n)
    t = df.get("license_terms", defaults).dropna().astype(float).mean()
    r = df.get("redistribution", defaults).dropna().astype(float).mean()
    e = df.get("ethical_reviews", defaults).dropna().astype(float).mean()
    return {"license_terms":t,"redistribution":r,"ethical_reviews":e}

def make_dim_funcs(dim_scores):
    return (
        lambda: dim_scores['dim1'],
        lambda df: calculate_medical_quality(df),
        lambda: dim_scores['dim3'],
        lambda: dim_scores['dim4'],
        lambda: dim_scores['dim5'],
        lambda df: calculate_usage_constraints(df)
    )

quality_profiles = {
    'high':    {'dim1':{'source':5,'audits':5,'collection':5},'dim2':{'completeness':5,'errors':4},'dim3':{'icd10_docs':5,'protocols':4},'dim4':{'freshness':4,'retention':5},'dim5':{'hipaa':5,'ethics':5}},
    'moderate':{'dim1':{'source':4,'audits':3,'collection':4},'dim2':{'completeness':3,'errors':3},'dim3':{'icd10_docs':3,'protocols':3},'dim4':{'freshness':3,'retention':3},'dim5':{'hipaa':4,'ethics':3}},
    'low':     {'dim1':{'source':2,'audits':1,'collection':2},'dim2':{'completeness':2,'errors':1},'dim3':{'icd10_docs':2,'protocols':1},'dim4':{'freshness':2,'retention':1},'dim5':{'hipaa':2,'ethics':2}}
}

def assign_pscore_templates(scenario):
    profiles={}
    N=len(CLIENT_IDS)
    if scenario=='high':
        for c in CLIENT_IDS: profiles[c]=make_dim_funcs(quality_profiles['high'])
    elif scenario=='varied':
        t1,t2=int(N*0.2),int(N*0.7)
        for i,c in enumerate(CLIENT_IDS):
            lvl = 'high' if i<t1 else 'moderate' if i<t2 else 'low'
            profiles[c]=make_dim_funcs(quality_profiles[lvl])
    else:  # low
        hc=random.choice(CLIENT_IDS)
        for c in CLIENT_IDS:
            lvl='high' if c==hc else 'low'
            profiles[c]=make_dim_funcs(quality_profiles[lvl])
    return profiles

def policy_pscore(score):
    if   score>=4.0: return 'Excellent','ACCEPT'
    elif score>=3.2: return 'Good','ACCEPT'
    elif score>=2.5: return 'Moderate','REVIEW'
    else:            return 'Poor','QUARANTINE'

def run_tadp_governance(client_data, client_profiles):
    central="./ledgers/central/central_ledger.csv"
    rows=[]

    for cid, df in client_data.items():
        fns=client_profiles[cid]
        # compute each dim
        dim_vals = [
            np.mean(list(fns[0]().values())) if isinstance(fns[0](), dict) else fns[0](),
            np.mean(list(fns[1](df).values())),
            np.mean(list(fns[2]().values())) if isinstance(fns[2](), dict) else fns[2](),
            np.mean(list(fns[3]().values())) if isinstance(fns[3](), dict) else fns[3](),
            np.mean(list(fns[4]().values())) if isinstance(fns[4](), dict) else fns[4](),
            np.mean(list(fns[5](df).values()))
        ]
        score = sum(WEIGHTS_PSCORE[f'dim{i+1}']*dim_vals[i] for i in range(6))
        label,action = policy_pscore(score)
        rows.append({
            'client':cid,'pscore':round(score,2),'action':action,
            **{f'dim{i+1}':round(dim_vals[i],2) for i in range(6)}
        })

    gov_df=pd.DataFrame(rows).set_index('client')
    display(gov_df)

    # Append to ledgers
    out=gov_df.reset_index()
    out['tx_id']        = [uuid.uuid4().hex for _ in range(len(out))]
    out['timestamp']    = datetime.utcnow().isoformat()
    out['record_count']= out['client'].map(lambda c: len(client_data[c]))
    out['version']      = 0
    for i,c in enumerate(out['client']):
        version_counters[c]=version_counters.get(c,0)+1
        out.at[i,'version']=version_counters[c]
    # reorder for write
    write=out[[
        "tx_id","timestamp","client","version","record_count",
        "dim1","dim2","dim3","dim4","dim5","dim6","pscore","action"
    ]]
    write.to_csv(central, mode='a', header=False, index=False)
    for _,r in write.iterrows():
        lp=f"./ledgers/local/{r.client}_ledger.csv"
        pd.DataFrame([r]).to_csv(lp, mode='a', header=not os.path.exists(lp), index=False)

    return gov_df
# ──────────────────────────────────────────────────────────────────────────────
# Cell 6 (updated): FLOWER CLIENT & SERVER SIMULATION
# ──────────────────────────────────────────────────────────────────────────────
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, model, X, y):
        self.cid   = cid
        self.model = model
        self.X     = X
        self.y     = y

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.X, self.y, epochs=DEFAULT_LOCAL_EPOCHS, verbose=0)
        return self.model.get_weights(), len(self.X), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, acc = self.model.evaluate(self.X, self.y, verbose=0)
        yhat = self.model.predict(self.X, verbose=0)
        metrics,_ = get_detailed_metrics(self.y, yhat)
        return loss, len(self.X), metrics

def client_fn(cid, clients, client_data, preprocessor):
    # cid comes in as a string index
    client_id = clients[int(cid)]
    raw = client_data[client_id]
    X = preprocessor.transform(raw.drop('readmitted', axis=1))
    y = raw['readmitted']
    model = make_model(X.shape[1], verbose=False)
    return FlowerClient(client_id, model, X, y).to_client()

def start_fl_simulation(clients, client_data, preprocessor, input_dim, X_test, y_test):
    def evaluate(rnd, params, cfg):
        # central evaluation
        m = make_model(input_dim, verbose=False)
        m.set_weights(params)
        p = m.predict(X_test, verbose=0)
        return 0, get_detailed_metrics(y_test, p)[0]

    strategy = fl.server.strategy.FedAvg(
        fraction_fit=DEFAULT_CLIENT_FRACTION,
        min_fit_clients=len(clients),
        min_available_clients=len(clients),
        evaluate_fn=evaluate
    )

    # **Pass** the chosen preprocessor into each client
    hist = fl.simulation.start_simulation(
        client_fn=lambda cid: client_fn(cid, clients, client_data, preprocessor),
        num_clients=len(clients),
        config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS_FL),
        strategy=strategy,
        client_resources={'num_cpus':1}
    )
    return hist


TensorFlow: 2.18.0, Flower: 1.18.0
⚠️ LEDGERS RESET: All previous ledger files will be deleted now. ⚠️
⬆️ Upload diabetes.csv


<>:81: DeprecationWarning: invalid escape sequence '\d'


Saving diabetic_data.csv to diabetic_data (3).csv


<ipython-input-22-8976fd03044e>:89: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace({'No':0,'Steady':1,'Up':1,'Down':1})



Client Data Distribution:
 • A: 45877 samples
 • B: 11133 samples
 • C: 11133 samples
 • D: 12233 samples
 • E: 21390 samples

🔍 Neural Network Overview:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


,Layer,Type,Output Shape,Params
0,dense_60,Dense,"(None, 128)",272256
1,dense_61,Dense,"(None, 32)",4128
2,dense_62,Dense,"(None, 3)",99


In [ ]:
# get any row before and after the transformer
raw_sample = raw_df.drop('readmitted', axis=1).iloc[[0]]
encoded_sample = preproc_all.transform(raw_sample)
print("Before:", raw_sample.shape, "→ After:", encoded_sample.shape)
assert encoded_sample.shape[1] == X_all.shape[1]


Before: (1, 49) → After: (1, 2126)


In [ ]:
df.shape

(21390, 50)

In [ ]:
# to Reset the Ledger
# metadata_ledgers = {client_id: [] for client_id in CLIENT_IDS}
# version_counters = {client_id: 0 for client_id in CLIENT_IDS}


# ──────────────────────────────────────────────
# Cell 7: SCENARIO EXECUTION (with FIXED VERSION HANDLING)
# ──────────────────────────────────────────────
from datetime import datetime
import uuid
import pandas as pd
import time
import numpy as np

saved_models = {}

# Define the specific scenarios we want to run
scenario_configs = {
    ("All Accepted", "Centralized Full"): {
        "profile": assign_pscore_templates('high'),
        "accept_all": True,
        "run_centralized": True,
        "run_federated": False,
        "is_tadp": False
    },
    ("All Accepted", "TADP Centralized"): {
        "profile": assign_pscore_templates('high'),
        "accept_all": True,
        "run_centralized": True,
        "run_federated": False,
        "is_tadp": True
    },
    ("All Accepted", "Federated Full"): {
        "profile": assign_pscore_templates('high'),
        "accept_all": True,
        "run_centralized": False,
        "run_federated": True,
        "is_tadp": False
    },
    ("All Accepted", "TADP Federated"): {
        "profile": assign_pscore_templates('high'),
        "accept_all": True,
        "run_centralized": False,
        "run_federated": True,
        "is_tadp": True
    },
    ("Varied Review", "TADP Centralized"): {
        "profile": assign_pscore_templates('varied'),
        "accept_all": False,
        "run_centralized": True,
        "run_federated": False,
        "is_tadp": True
    },
    ("Varied Review", "TADP Federated"): {
        "profile": assign_pscore_templates('varied'),
        "accept_all": False,
        "run_centralized": False,
        "run_federated": True,
        "is_tadp": True
    },
    ("Single Dataset Accepted", "TADP Centralized"): {
        "profile": assign_pscore_templates('low'),
        "accept_all": False,
        "run_centralized": True,
        "run_federated": False,
        "is_tadp": True
    },
    ("Single Dataset Accepted", "TADP Federated"): {
        "profile": assign_pscore_templates('low'),
        "accept_all": False,
        "run_centralized": False,
        "run_federated": True,
        "is_tadp": True
    }
}

METRICS = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
results = {}

def get_fl_metrics(hist, metrics_list):
    results = {}
    for m in metrics_list:
        if hasattr(hist, "metrics_centralized") and m in hist.metrics_centralized:
            vals = hist.metrics_centralized[m]
            if isinstance(vals, list) and len(vals) > 0:
                results[m] = float(vals[-1][1])
            else:
                results[m] = float('nan')
        else:
            results[m] = float('nan')
    return results

# Function to record a manual review decision (does NOT increment version!)
def record_review_decision(client_id, version, decision, ledger_dir="./ledgers/local/"):
    ledger_path = f"{ledger_dir}/{client_id}_ledger.csv"
    df = pd.read_csv(ledger_path)
    last_entries = df[df['version'] == version]
    if last_entries.empty:
        print(f"[Warning] No entry found for client {client_id} version {version} in ledger!")
        return
    last_entry = last_entries.iloc[-1]
    new_entry = last_entry.copy()
    new_entry['action'] = decision
    new_entry['timestamp'] = datetime.utcnow().isoformat()
    new_entry['tx_id'] = uuid.uuid4().hex
    # VERSION IS NOT CHANGED
    df = pd.concat([df, pd.DataFrame([new_entry])], ignore_index=True)
    df.to_csv(ledger_path, index=False)

    # --- Also append this override to the central ledger ---
    central_path = "./ledgers/central/central_ledger.csv"
    central_df = pd.read_csv(central_path)
    # You may need to add dummy values for columns not in the local ledger
    # (Example assumes all required columns are present)
    cols = central_df.columns.tolist()
    # Build a new row for central ledger
    new_central = {}
    for col in cols:
        if col in new_entry:
            new_central[col] = new_entry[col]
        else:
            # e.g., fill with NaN or repeat from last_entry
            if col in last_entry:
                new_central[col] = last_entry[col]
            else:
                new_central[col] = np.nan
    central_df = pd.concat([central_df, pd.DataFrame([new_central])], ignore_index=True)
    central_df.to_csv(central_path, index=False)


# Main Scenario Execution Loop
for scenario_name, config in scenario_configs.items():
    display_name = f"{scenario_name[0]} - {scenario_name[1]}"
    print(f"\n===== Scenario: {display_name} =====")

    # Run governance if this is a TADP scenario
    if config["is_tadp"] or True:  # <-- This ensures ALL scenarios enter this block
        gov_df = run_tadp_governance(client_data_splits, config["profile"])
        if config["accept_all"]:
            accepted = CLIENT_IDS.copy()
            review_decisions = pd.Series(['ACCEPT'] * len(CLIENT_IDS), index=CLIENT_IDS)
        else:
            # For non-accept-all scenarios, process REVIEW clients
            accepted = gov_df[gov_df['action'] == 'ACCEPT'].index.tolist()
            review_decisions = gov_df['action'].copy()

            # Handle manual review for Varied Review scenario
            if scenario_name[0] == 'Varied Review':
                review_clients = gov_df[gov_df['action'] == 'REVIEW'].index.tolist()
                for cid in review_clients:
                    version = version_counters.get(cid, 1)  # Do not increment here!
                    ans = input(f"Include client {cid} (version {version})? [y/n]: ").strip().lower()
                    if ans == 'y':
                        accepted.append(cid)
                        review_decisions[cid] = 'ACCEPT'
                        record_review_decision(cid, version, 'ACCEPT')
                    else:
                        review_decisions[cid] = 'QUARANTINE'
                        record_review_decision(cid, version, 'QUARANTINE')
    else:
        # For non-TADP scenarios, use all clients
        accepted = CLIENT_IDS.copy()
        gov_df = None

    # Prepare data for this scenario
    if config["is_tadp"] and accepted:
        X_scenario, X_scenario_test, y_scenario, y_scenario_test, preprocessor_scenario, _ = preprocess_data(
            {c: client_data_splits[c] for c in accepted}
        )
    else:
        # Use all data for non-TADP scenarios
        X_scenario, X_scenario_test, y_scenario, y_scenario_test = X_all, X_all_test, y_all, y_all_test
        preprocessor_scenario = preproc_all

    # Run the appropriate training based on configuration
    scenario_results = {}

    # Centralized training if configured
    if config["run_centralized"]:
        t0 = time.time()
        model = make_model(X_scenario.shape[1], verbose=False)
        model.fit(X_scenario, y_scenario, epochs=5, batch_size=32, verbose=0)

        # --- Save model & test set for later softmax output analysis ---
        saved_models[scenario_name] = {
            "model": model,
            "X_test": X_scenario_test,
            "y_test": y_scenario_test
    }
        train_time = time.time() - t0
        metrics, _ = get_detailed_metrics(y_scenario_test, model.predict(X_scenario_test))

        scenario_results.update({
            'param_count': model.count_params(),
            'train_time': train_time,
            'metrics': metrics,
            'model_type': 'centralized',
            'is_tadp': config["is_tadp"]
        })


    # Federated training if configured
    elif config["run_federated"]:
        t0 = time.time()
        hist = start_fl_simulation(
            accepted, client_data_splits, preprocessor_scenario,
            X_scenario.shape[1], X_scenario_test, y_scenario_test
        )
        train_time = time.time() - t0
        metrics = get_fl_metrics(hist, METRICS)

        scenario_results.update({
            'param_count': make_model(X_scenario.shape[1], verbose=False).count_params(),
            'train_time': train_time,
            'metrics': metrics,
            'model_type': 'federated',
            'is_tadp': config["is_tadp"]
        })

    # Store results
    results[scenario_name] = {
        **scenario_results,
        'accepted_clients': accepted,
        'gov_df': gov_df
    }

# Print summary of results
print("\n=== Scenario Results Summary ===")
for scenario_name, res in results.items():
    print(f"\n{scenario_name[0]} - {scenario_name[1]}:")
    print(f"  Model Type: {res['model_type'].capitalize()}")
    print(f"  TADP: {'Yes' if res['is_tadp'] else 'No'}")
    print(f"  Accepted Clients: {len(res['accepted_clients'])}/{len(CLIENT_IDS)}")  # Fixed the f-string here
    print(f"  Parameters: {res['param_count']}")
    print(f"  Training Time: {res['train_time']:.2f}s")
    print("  Metrics:")
    for metric, value in res['metrics'].items():
        print(f"    {metric}: {value:.4f}")



===== Scenario: All Accepted - Centralized Full =====


,pscore,action,dim1,dim2,dim3,dim4,dim5,dim6
client,,,,,,,,
A,4.2,ACCEPT,5.0,1.64,4.5,4.5,5.0,3.0
B,4.2,ACCEPT,5.0,1.64,4.5,4.5,5.0,3.0
C,4.2,ACCEPT,5.0,1.64,4.5,4.5,5.0,3.0
D,4.2,ACCEPT,5.0,1.64,4.5,4.5,5.0,3.0
E,4.2,ACCEPT,5.0,1.64,4.5,4.5,5.0,3.0


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


637/637 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

===== Scenario: All Accepted - TADP Centralized =====


,pscore,action,dim1,dim2,dim3,dim4,dim5,dim6
client,,,,,,,,
A,4.2,ACCEPT,5.0,1.64,4.5,4.5,5.0,3.0
B,4.2,ACCEPT,5.0,1.64,4.5,4.5,5.0,3.0
C,4.2,ACCEPT,5.0,1.64,4.5,4.5,5.0,3.0
D,4.2,ACCEPT,5.0,1.64,4.5,4.5,5.0,3.0
E,4.2,ACCEPT,5.0,1.64,4.5,4.5,5.0,3.0


...

287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

===== Scenario: Single Dataset Accepted - TADP Federated =====


,pscore,action,dim1,dim2,dim3,dim4,dim5,dim6
client,,,,,,,,
A,4.20,ACCEPT,5.00,1.64,4.5,4.5,5.0,3.0
B,1.86,QUARANTINE,1.67,1.64,1.5,1.5,2.0,3.0
C,1.86,QUARANTINE,1.67,1.64,1.5,1.5,2.0,3.0
D,1.86,QUARANTINE,1.67,1.64,1.5,1.5,2.0,3.0
E,1.86,QUARANTINE,1.67,1.64,1.5,1.5,2.0,3.0


Setting `min_available_clients` lower than `min_fit_clients` or
`min_evaluate_clients` can cause the server to fail when there are too few clients
connected to the server. `min_available_clients` must be set to a value larger
than or equal to the values of `min_fit_clients` and `min_evaluate_clients`.

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=10, no round_timeout
2025-05-28 16:24:41,871	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'object_store_memor


=== Scenario Results Summary ===

All Accepted - Centralized Full:
  Model Type: Centralized
  TADP: No
  Accepted Clients: 5/5
  Parameters: 276483
  Training Time: 81.29s
  Metrics:
    accuracy: 0.5830
    precision: 0.5600
    recall: 0.5830
    f1: 0.5502
    roc_auc: 0.6605

All Accepted - TADP Centralized:
  Model Type: Centralized
  TADP: Yes
  Accepted Clients: 5/5
  Parameters: 276483
  Training Time: 84.74s
  Metrics:
    accuracy: 0.5830
    precision: 0.5600
    recall: 0.5830
    f1: 0.5502
    roc_auc: 0.6605

All Accepted - Federated Full:
  Model Type: Federated
  TADP: No
  Accepted Clients: 5/5
  Parameters: 276483
  Training Time: 697.32s
  Metrics:
    accuracy: 0.6999
    precision: 0.6896
    recall: 0.6999
    f1: 0.6842
    roc_auc: 0.8166

All Accepted - TADP Federated:
  Model Type: Federated
  TADP: Yes
  Accepted Clients: 5/5
  Parameters: 276483
  Training Time: 678.01s
  Metrics:
    accuracy: 0.6950
    precision: 0.6860
    recall: 0.6950
    f1: 0.677

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [ ]:
# ─────────────────────────────────────────────
# Cell 7.10: Professional Central Ledger Summary (Q1 Level)
# ─────────────────────────────────────────────
import pandas as pd
from IPython.display import display, HTML

# Read central ledger
central_ledger_path = './ledgers/central/central_ledger.csv'
df = pd.read_csv(central_ledger_path)

def highlight_action(val):
    if isinstance(val, str):
        if val.upper() == "ACCEPT":
            return 'background-color: #d4edda; color: #155724; font-weight:bold'   # Light green
        elif val.upper() == "REVIEW":
            return 'background-color: #fff3cd; color: #856404; font-weight:bold'   # Light orange/yellow
        elif val.upper() == "QUARANTINE":
            return 'background-color: #f8d7da; color: #721c24; font-weight:bold'   # Light red
    return ''

if df.empty:
    print("Central ledger is empty.")
else:
    summary = []
    for (client, version), group in df.groupby(['client', 'version']):
        group = group.sort_values('timestamp')
        first = group.iloc[0]
        last = group.iloc[-1]
        summary.append({
            'client': client,
            'version': version,
            'timestamp': first['timestamp'],
            'tx_id': first['tx_id'],
            'record_count': first['record_count'],
            'pscore': first['pscore'],
            'initial_action': first['action'],
            'final_action': last['action']
        })
    ledger_summary = pd.DataFrame(summary)
    ledger_summary = ledger_summary.sort_values(['client', 'version']).reset_index(drop=True)
    ledger_summary['was_overridden'] = ledger_summary['initial_action'] != ledger_summary['final_action']

    # Highlighting for publication
    def highlight_row(row):
        # Entire row highlight if overridden
        if row['was_overridden']:
            return ['background-color: #fff9c4; font-weight:bold']*len(row)
        else:
            return ['']*len(row)

    styler = ledger_summary.style \
        .applymap(highlight_action, subset=['initial_action', 'final_action']) \
        .apply(highlight_row, axis=1) \
        .format({'pscore': '{:.2f}'}) \
        .set_caption("Table: Central Aggregated Metadata Ledger. Initial and Final actions are color-coded. Rows highlighted if an override occurred (admin intervention).")

    print("\n======= CENTRAL AGGREGATED METADATA LEDGER (Initial & Final Action) =======\n")
    display(styler)

    # Display override rows only, if any
    if ledger_summary['was_overridden'].any():
        override_styler = ledger_summary[ledger_summary['was_overridden']].style \
            .applymap(highlight_action, subset=['initial_action', 'final_action']) \
            .format({'pscore': '{:.2f}'}) \
            .set_caption("Table: Entries Where Admin Review Overrode the Initial Action")
        print("\n--- Entries Where Review Overrode the Initial Action ---")
        display(override_styler)



======= CENTRAL AGGREGATED METADATA LEDGER (Initial & Final Action) =======



<ipython-input-26-3a8d196f9f52>:52: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(highlight_action, subset=['initial_action', 'final_action']) \


,client,version,timestamp,tx_id,record_count,pscore,initial_action,final_action,was_overridden
0,A,1,2025-05-28T15:49:18.491195,5c78870047c54f0b8ffd66fd298288ec,45877,4.20,ACCEPT,ACCEPT,False
1,A,2,2025-05-28T15:50:42.859599,500e231728314d669b22cafeb97974d1,45877,4.20,ACCEPT,ACCEPT,False
2,A,3,2025-05-28T15:52:13.070791,977e76c6388f4024ba43dae3804a9951,45877,4.20,ACCEPT,ACCEPT,False
3,A,4,2025-05-28T16:03:50.743019,af98ed7bb0f94e578d3b4a2ac1b922f1,45877,4.20,ACCEPT,ACCEPT,False
4,A,5,2025-05-28T16:15:13.634123,959bd2d121b34f5ba133b856be56758a,45877,4.20,ACCEPT,ACCEPT,False
5,A,6,2025-05-28T16:16:24.249693,3231febfb6ef457f90672ab1b9dcda68,45877,4.20,ACCEPT,ACCEPT,False
6,A,7,2025-05-28T16:23:49.240081,17bb4f375f324b538b93d9c5d04ea2ed,45877,4.20,ACCEPT,ACCEPT,False
7,A,8,2025-05-28T16:24:35.877429,8d97a4e9374d49edab4f242bdb31af23,45877,4.20,ACCEPT,ACCEPT,False
8,B,1,2025-05-28T15:49:18.491195,a4ee18ad9dbd401f8b34dee94d6ad8c8,11133,4.20,ACCEPT,ACCEPT,False
9,B,2,2025-05-28T15:50:42.859599,50e480e5d7f643d991f2a3a93b3254c2,11133,4.20,ACCEPT,ACCEPT,False



--- Entries Where Review Overrode the Initial Action ---


<ipython-input-26-3a8d196f9f52>:63: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(highlight_action, subset=['initial_action', 'final_action']) \


,client,version,timestamp,tx_id,record_count,pscore,initial_action,final_action,was_overridden
12,B,5,2025-05-28T16:15:13.634123,98e38b0b1a034272a89df500472f80c4,11133,3.11,REVIEW,ACCEPT,True
13,B,6,2025-05-28T16:16:24.249693,d618ed26c2354727999309cc4be7495f,11133,3.11,REVIEW,ACCEPT,True
20,C,5,2025-05-28T16:15:13.634123,cb18a15236f94696a7d14b46192f6317,11133,3.11,REVIEW,ACCEPT,True
21,C,6,2025-05-28T16:16:24.249693,92c50cf8253d4a86a6df394181f4a7a5,11133,3.11,REVIEW,ACCEPT,True


In [ ]:
# ──────────────────────────────────────────────────────────────────────────────
# Cell 8: PERFORMANCE & SUSTAINABILITY ANALYSIS (For 8 Specific Scenarios)
# ──────────────────────────────────────────────────────────────────────────────

# Constants for calculations
POWER_W = 45                # Average system power in Watts
COST_PER_KWH = 0.2          # Electricity price (USD per kWh)
CARBON_INTENSITY = 0.475    # kg CO2 per kWh

def calculate_communication_cost(params, rounds):
    """Calculate communication cost in MB (4 bytes per param, up/down per round)"""
    return params * 4 * 2 * rounds / (1024**2)

def calculate_energy_cost(time_s, devices=1):
    """Calculate energy consumption, cost, and CO2 emissions"""
    joules = POWER_W * time_s * devices
    kwh = joules / 3.6e6
    cost = kwh * COST_PER_KWH
    co2 = kwh * CARBON_INTENSITY
    return kwh, cost, co2

# Prepare performance analysis for all 8 scenarios
performance_data = []
for scenario_name, res in results.items():
    scenario, approach = scenario_name  # Unpack the scenario tuple

    # Common metrics for all approaches
    metrics = {
        'Scenario': scenario,
        'Approach': approach,
        **res['metrics'],
        'Parameters': res['param_count'],
        'Time_s': res['train_time'],
        'Clients': len(res['accepted_clients'])
    }

    # Add approach-specific calculations
    if 'Centralized' in approach:
        # Centralized approaches have no communication cost
        kwh, cost, co2 = calculate_energy_cost(res['train_time'])
        metrics.update({
            'Comm_MB': 0,
            'Energy_kWh': kwh,
            'Cost_USD': cost,
            'CO2_kg': co2,
            'Rounds': 1
        })
    else:
        # Federated approaches include communication costs
        comm = calculate_communication_cost(res['param_count'], NUM_ROUNDS_FL)
        kwh, cost, co2 = calculate_energy_cost(res['train_time'], len(res['accepted_clients']))
        metrics.update({
            'Comm_MB': comm,
            'Energy_kWh': kwh,
            'Cost_USD': cost,
            'CO2_kg': co2,
            'Rounds': NUM_ROUNDS_FL
        })

    performance_data.append(metrics)

# Create final DataFrame with all 8 scenarios
analysis_df = pd.DataFrame(performance_data)

# Reorder columns for better readability
column_order = [
    'Scenario', 'Approach', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc',
    'Parameters', 'Clients', 'Time_s', 'Comm_MB', 'Energy_kWh', 'Cost_USD', 'CO2_kg', 'Rounds'
]
analysis_df = analysis_df[column_order]

print("📊 Performance Analysis for All 8 Scenarios:")
display(analysis_df)

📊 Performance Analysis for All 8 Scenarios:


,Scenario,Approach,accuracy,precision,recall,f1,roc_auc,Parameters,Clients,Time_s,Comm_MB,Energy_kWh,Cost_USD,CO2_kg,Rounds
0,All Accepted,Centralized Full,0.583030,0.559998,0.583030,0.550168,0.660533,276483,5,81.293794,0.000000,0.001016,0.000203,0.000483,1
1,All Accepted,TADP Centralized,0.583030,0.559998,0.583030,0.550168,0.660533,276483,5,84.741430,0.000000,0.001059,0.000212,0.000503,1
2,All Accepted,Federated Full,0.699912,0.689633,0.699912,0.684167,0.816555,276483,5,697.319612,21.093979,0.043582,0.008716,0.020702,10
3,All Accepted,TADP Federated,0.694999,0.685978,0.694999,0.677082,0.814101,276483,5,678.007852,21.093979,0.042375,0.008475,0.020128,10
4,Varied Review,TADP Centralized,0.558735,0.557778,0.558735,0.540706,0.658048,262019,3,43.880427,0.000000,0.000549,0.000110,0.000261,1
5,Varied Review,TADP Federated,0.761024,0.758314,0.761024,0.753407,0.870855,262019,3,438.314369,19.990463,0.016437,0.003287,0.007807,10
6,Single Dataset Accepted,TADP Centralized,0.573561,0.536924,0.573561,0.537440,0.651435,246019,1,43.339220,0.000000,0.000542,0.000108,0.000257,1
7,Single Dataset Accepted,TADP Federated,0.898976,0.899776,0.898976,0.898375,0.976839,246019,1,205.314481,18.769760,0.002566,0.000513,0.001219,10
